In [4]:
import numpy as np
import networkx as nx
import random
import matplotlib.pyplot as plt

def create_random_graph(num_nodes, prob_edge):
    # create random graph with specified number of nodes and probability of an edge between two nodes
    graph = nx.Graph()
    for i in range(num_nodes):
        for j in range(i+1, num_nodes):
            if random.random() < prob_edge:
                graph.add_edge(i, j)
    return graph

def initialize_partition(num_nodes):
    # create initial partition where each node is in its own community
    partition = {}
    for i in range(num_nodes):
        partition[i] = i
    return partition

def calculate_modularity(graph, partition, m):
    # calculate modularity of partition in graph
    q = 0
    for i in partition.keys():
        ki = len(list(graph.neighbors(i)))
        for j in partition.keys():
            kj = len(list(graph.neighbors(j)))
            if partition[i] == partition[j]:
                Aij = 1
            else:
                Aij = 0
            q += (Aij - ki*kj/(2*m))            
    return q/(2*m)

def leiden_algorithm(graph, num_nodes, prob_edge):
    # initialize variables
    m = len(list(graph.edges()))
    partition = initialize_partition(num_nodes)
    modularity = calculate_modularity(graph, partition, m)
    best_partition = partition.copy()
    best_modularity = modularity
    
    # perform Louvain algorithm until no further improvement in modularity
    while True:
        # perform first phase of Louvain algorithm
        improvement = True
        while improvement:
            improvement = False
            for i in range(num_nodes):
                initial_community = partition[i]
                best_community = initial_community
                max_delta_q = 0
                for j in graph.neighbors(i):
                    if partition[j] != initial_community:
                        delta_q = 2*(nx.algorithms.community.quality.modularity_gain(graph, partition, initial_community, j) - nx.algorithms.community.quality.modularity_gain(graph, partition, initial_community, initial_community))
                        if delta_q > max_delta_q:
                            max_delta_q = delta_q
                            best_community = partition[j]
                if best_community != initial_community:
                    improvement = True
                    partition[i] = best_community
            if improvement:
                community_counts = {}
                for i in partition.values():
                    if i in community_counts:
                        community_counts[i] += 1
                    else:
                        community_counts[i] = 1
                for i in partition.keys():
                    if community_counts[partition[i]] == 1:
                        partition[i] = i
        
        # perform second phase of Louvain algorithm
        new_graph = nx.Graph()
        community_mapping = {}
        community_index = 0
        for i in partition.keys():
            community = partition[i]
            if community not in community_mapping:
                community_mapping[community] = community_index
                community_index += 1
            new_graph.add_node(community_mapping[community])
        for i, j in graph.edges():
            ci = partition[i]
            cj = partition[j]
            if ci != cj:
                if new_graph.has_edge(community_mapping[ci], community_mapping[cj]):
                    new_graph[community_mapping[ci]][community_mapping[cj]]['weight'] += 1
                else:
                    new_graph.add_edge(community_mapping[ci], community_mapping[cj], weight=1)
        
        partition = initialize_partition
num_nodes = 50
prob_edge = 0.1

graph = create_random_graph(num_nodes, prob_edge)

final_graph, final_partition, modularity = leiden_algorithm(graph, num_nodes, prob_edge)

# plot final partition
pos = nx.spring_layout(final_graph)
colors = [final_partition[n] for n in final_graph.nodes()]
nx.draw(final_graph, pos, node_color=colors, with_labels=True)
plt.show()

print("Final modularity:", modularity)


AttributeError: module 'networkx.algorithms.community.quality' has no attribute 'modularity_gain'